In [9]:
import pandas as pd

# 替换为你的 CSV 文件路径
csv_file_path = 'split_file_1.csv'

# 读取 CSV 文件
df = pd.read_csv(csv_file_path)

# 将字符串转换为列表，如果不是字符串则将其转换为空列表
df['contexts'] = df['contexts'].apply(lambda x: [x])


df


FileNotFoundError: [Errno 2] No such file or directory: 'split_file_1.csv'

In [10]:
from datasets import Dataset

# 创建字典
data_dict = {
    'question': df['question'].tolist(),
    'contexts': df['contexts'].tolist(),
    'answer': df['answer'].tolist(),
    'ground_truth': df['ground_truth'].tolist()  # 如果有其他列，类似地处理
}

# 使用字典创建数据集
hf_dataset = Dataset.from_dict(data_dict)
hf_dataset

Dataset({
    features: ['question', 'contexts', 'answer', 'ground_truth'],
    num_rows: 10
})

In [11]:
from ragas.metrics import (
    # answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
    context_entity_recall,
    answer_correctness,
    answer_similarity,
)

In [12]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# 为embedding设置独立api
openai_key_embedding = 'sk-S52sOieQg04WZzEtNa4gKt52RcJQ3qh6ac8GZfWQhBqQskUg'
# 为chatbot设置独立api，重定向至deepseek
# deepseek_key = 'sk-aff8187171d643828da2280bf1846bc0'

llm = ChatOpenAI(model="gpt-4o-mini", base_url="https://api.openai-proxy.org/v1", api_key=openai_key_embedding)

embeddings = OpenAIEmbeddings(model="text-embedding-3-small", base_url="https://api.openai-proxy.org/v1", api_key=openai_key_embedding)

In [13]:
from ragas import evaluate
from ragas.run_config import RunConfig

result = evaluate(
    hf_dataset,
    metrics=[
    # answer_relevancy,
    # faithfulness,
    # context_recall,
    # context_precision,
    # context_entity_recall,
    answer_correctness,
    answer_similarity,
    ],
    llm=llm,
    embeddings=embeddings,
    run_config=RunConfig(max_workers=16, max_retries=3, max_wait=30)
)

result

Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]

{'answer_correctness': 0.9422, 'answer_similarity': 0.7688}

In [14]:
df = result.to_pandas()
df

,question,contexts,answer,ground_truth,answer_correctness,answer_similarity
0,二禅的特点是什么，以及它与初禅有何不同？,[[�；在坐禅时，进入初禅的时候，身体会有很舒服的感觉，那就是乐。所以进入初禅也称为‘离生喜...,二禅是佛教修行中的一个深入阶段，与初禅相比，二禅的特点主要体现在专注的深度、体验的纯净度以及...,二禅的特点包括无觉无观、内心平静、清净、无眼、耳、身识，唯有意识，以及具有内静、喜、乐、一心...,0.945618,0.782544
1,为什么说涅槃并不是一种境界？,[[解所执著的渣滓皆丢光。如果追求解脱者想‘得到’解脱，那就是错了，因为解脱是‘什么都得不到...,涅槃在佛教中是一个极为深奥的概念，它通常被理解为超越生死轮回的终极解脱状态。涅槃并不是一种境...,涅槃并不是一种境界，因为如果涅槃是一种境界，能进入那种境界，必然能够出，有进、有出，就是有生...,0.932793,0.731171
2,"""在观察法念处时，如何通过观察受的生起与灭掉来消除贪嗔痴？",[[�用心观察，慢慢的就能掌握消除五盖的方法，它就会消除掉，那我们就不会再怀疑佛法，对它深信...,在观察法念处时，通过觉知感受的生起与灭去，修行者可以洞察感受的无常和苦的本质，从而减少对感受...,在观察法念处时，通过观察受的生起与灭掉，可以消除贪嗔痴。一旦你看到受的生起、再灭掉，那么我们...,0.903793,0.615204
3,“在佛教冥想中，修习‘观心无常’的目的是什么？”,[[�、香、味、触、法六尘，称为十八界。我们个别观察它，里面没有一个我，这十八界合起来的一个...,在佛教冥想中，修习“观心无常”的目的是帮助修行者认识到心念的无常性，从而减少对心念的执著，培...,佛教冥想中实践“观心无常”的目的在于认识到心念始终处于变化之中，并无恒定不变的状态，从而有助...,0.956631,0.826628
4,"""为什么皈依后应该亲近善知识？",[[缘使得我们认识到自心，那是非常可贵的，所以我们看到佛、法、僧，要生起如获至宝之想。\n\...,皈依后亲近善知识是因为他们能提供正确的修行指导，帮助理解佛教教义，避免走入歧途；通过交流学习...,皈依后应该亲近善知识，因为亲近善知识可以帮助我们听闻正法，生起正见，并且依教奉行，以僧为榜样...,0.940159,0.760636
5,佛陀时代，化相三宝的僧宝包括哪些成员，他们的修行目的是什么？,[[是和合众的意思。奉行佛陀教法的佛陀出家弟子，这些实践佛法的出家弟子都称为僧，他们是比丘、...,在佛陀时代，化相三宝中的僧宝指的是佛陀的弟子，即比丘和比丘尼，他们是出家修行的僧侣。他们的修...,佛陀时代，化相三宝的僧宝包括无数出家的凡圣弟子，他们的修行目的是依据佛法修行，结果解脱证果，...,0.921904,0.687550
6,如何通过六根与六尘的接触减少贪嗔痴？,[[�用心观察，慢慢的就能掌握消除五盖的方法，它就会消除掉，那我们就不会再怀疑佛法，对它深信...,通过六根与六尘的接触减少贪嗔痴，关键在于保持觉知，观察接触带来的感受和反应，认识到它们的无常...,通过观察六根与六尘的接触，了了分明受的生起和灭掉，可以减少贪嗔痴。例如，当有人骂你时，通过观...,0.943976,0.775940
7,在菩萨道修行中，哪些布施与持戒行为能助修行者超越生死轮回，实现解脱，并为何这些行为被称为波罗蜜？,[[�闻到正法，了解人生是很苦恼的，同时也使他们懂得解脱的道理，能令他们走上解脱之道，这三种...,在菩萨道修行中，布施、持戒、忍辱、精进、禅定、智慧这六种行为被称为波罗蜜（Paramita）...,在菩萨道修行中，闻到正法、了解人生苦恼、懂得解脱道理的布施，以及持戒中的律仪戒、摄善法戒、饶...,0.964833,0.859465
8,在修行过程中，如何通过经行和静坐来达到心念了了分明的状态？,[[�先修行者要放下一切事物，住在庙宇中，然后跟随一位有资格的导师学习。每天起来，坐禅与经行...,通过经行和静坐来达到心念了了分明的状态，可以在经行时专注于步伐和身体移动，感受脚与地面的接触...,在修行过程中，通过经行和静坐来达到心念了了分明的状态，主要是通过观察身体的一举一动和呼吸的出...,0.957411,0.829645
9,在佛教修行中，对佛、法、僧以及戒律的绝对信心如何影响一个人证悟初果？,[[何人都不可毁坏。假如一个人对佛、法、僧以及戒有绝对的信心，一点儿也不怀疑，那他就是断了疑...,在佛教修行中，对佛、法、僧以及戒律的绝对信心是证悟初果的重要因素。这种信心能够激励修行者遵循...,在佛教修行中，对佛、法、僧以及戒律的绝对信心可以断除疑结，使一个人证悟到初果。如果学佛者对这...,0.954796,0.819179


In [16]:
df.to_csv("eval_by_4omini.csv", index=False)